# **PressPlay**

In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re

In [2]:
options = Options()
options.add_argument("--disable-notifications")

df_url = pd.DataFrame({'c_url' : []})
count = 0

for i in range(70):
    chrome = webdriver.Chrome('./chromedriver')
    chrome.get("https://www.pressplay.cc/project?page=" + str(i + 1))
    soup = BeautifulSoup(chrome.page_source, 'html.parser')

    for link in soup.find_all('a'):
        if not link.get('href').find('/project/'):
            df_url.loc[count] = link.get('href')[9:-6]
            count += 1
    chrome.close()

df_url = df_url.drop_duplicates()
df_url

,c_url
0,51473D7D4F8CA7CF9746BB56F6DB0286
3,DE447F29E3CE52AA2587E348E180A95C
6,B470FFB794729BF00A941766483B0251
9,2EB38014AB60773061B102547B850E21
12,4F0C5A0FBABB4DFF1DB7527BB76C7649
...,...
2429,F387AC3D2166AADB2AF9350141FACE5B
2432,2A132C8C5B33C50EF99FBCBF07975482
2438,35B81274FA6B4E2E2932085202E6056D
2441,36E5E7753CB40D8E7A8B5792FCCB0480


In [11]:
df_url_reset = pd.DataFrame(df_url['c_url'])
df_url_reset = df_url_reset.reset_index(drop=True)
df_url_reset

,c_url
0,51473D7D4F8CA7CF9746BB56F6DB0286
1,DE447F29E3CE52AA2587E348E180A95C
2,B470FFB794729BF00A941766483B0251
3,2EB38014AB60773061B102547B850E21
4,4F0C5A0FBABB4DFF1DB7527BB76C7649
...,...
719,F387AC3D2166AADB2AF9350141FACE5B
720,2A132C8C5B33C50EF99FBCBF07975482
721,35B81274FA6B4E2E2932085202E6056D
722,36E5E7753CB40D8E7A8B5792FCCB0480


In [40]:
df_courses = pd.DataFrame({'c_id' : [], 'c_name' : [], 'w_id' : [], 'c_description' : [], 'c_price' : [], 'c_total_time' : [], 't_id' : [], 'c_start_time' : [], \
                           'c_buyers' : [], 'cg_id' : [], 'c_url' : [], 'c_image_url' : [], 'status_id' : []})
df_teachers = pd.DataFrame({'t_id' : [], 't_name' : [], 't_introduction' : []})
df_fundings = pd.DataFrame({'f_id' : [], 'f_due_date' : [], 'f_process' : [], 'c_id' : []})

In [ ]:
for i in range(724):
    chrome = webdriver.Chrome('./chromedriver')
    chrome.get("https://www.pressplay.cc/project/" + df_url_reset['c_url'][i] + "/about")
    soup = BeautifulSoup(chrome.page_source, 'html.parser')
    c_id = 'pressplay_' + str(i + 1)
    if not soup.find('h1'):
        df_courses.loc[i] = {'c_id' : c_id, 'c_url' : "https://www.pressplay.cc/project/" + df_url_reset['c_url'][i] + "/about"}
        print(str(i) + ' is a restricted course')
    else:
        c_name = soup.find('h1').string
        if not soup.find(attrs={'name':'description'}):
            c_description = ''
        else:
            c_description = soup.find(attrs={'name':'description'})['content']
        if not soup.find('div', class_='project-info-price'):
            c_price = 0
        else:
            c_price = int(str(soup.find('div', class_='project-info-price').div.string).replace(',', '').replace('NT', '').replace('起', ''))
        if not soup.find('div', class_='project-info-meta-item'):
            c_buyers = 0
        else:
            c_buyers = int(str(soup.find('div', class_='project-info-meta-item').span.string).replace(',', ''))
        for link in soup.find_all('a'): # 課堂類別
            if not link.get('href').find('/project?type='):
                temp = link.string
                break 
        if temp == '語言學習':
            cg_id = '01'
        elif temp == '廣告行銷':
            cg_id = '06'
        elif temp == '投資理財':
            cg_id = '07'
        elif temp == '職場技能':
            cg_id = '08'
        else:
            cg_id = '09'
        c_url = "https://www.pressplay.cc/project/" + df_url_reset['c_url'][i] + "/about"
        c_img_url = soup.find(attrs={'property':'og:image'})['content']
        t_name = soup.find(attrs={'name':'author'})['content']

        if soup.find('div', class_='count-down-progress-text'):
            f_process = float(str(soup.find('div', class_='count-down-progress-text').span.string[5:-1])) / 100
        if soup.find('div', class_='count-down-text'):
            f_due_date = soup.find('div', class_='count-down-text').span.string[2:]
            df_fundings.loc[i] = {'f_due_date' : f_due_date, 'f_process' : f_process, 'c_id' : c_id}
        if soup.find('span', string=re.compile('^預計')):
            c_start_time = soup.find('span', string=re.compile('^預計')).string[3:-3]
            df_courses.loc[i] = {'c_id' : c_id, 'c_name' : c_name, 'c_description' : c_description, 'c_price' : c_price, 't_id' : int(i + 1), 'c_start_time' : c_start_time, 'c_buyers' : c_buyers, 'cg_id' : cg_id, 'c_url' : c_url, 'c_image_url' : c_img_url}
        else:
            df_courses.loc[i] = {'c_id' : c_id, 'c_name' : c_name, 'c_description' : c_description, 'c_price' : c_price, 't_id' : int(i + 1), 'c_buyers' : c_buyers, 'cg_id' : cg_id, 'c_url' : c_url, 'c_image_url' : c_img_url}
        df_teachers.loc[i] = {'t_id' : int(i + 1), 't_name' : t_name}
    chrome.close()

0 is a restricted course


In [45]:
df_courses

,c_id,c_name,w_id,c_description,c_price,c_total_time,t_id,c_start_time,c_buyers,cg_id,c_url,c_image_url,status_id
0,pressplay_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.pressplay.cc/project/51473D7D4F8CA...,NaN,NaN
1,pressplay_2,烘焙實驗室｜在家做 16 款職人級麵包,NaN,吐司Ｘ鹽可頌 X 貝果 Ｘ 佛卡夏， 搭配各式風味變化，一次學會 16 款職人麵包！「發酵」...,2580.0,NaN,2.0,2021/12/27,533.0,09,https://www.pressplay.cc/project/DE447F29E3CE5...,https://static.pressplay.cc/static/uploads/pro...,NaN
2,pressplay_3,【療癒相談室】12堂課解鎖生命地圖 開啟能量之旅,NaN,在變動的生活與狀態，\n讓ELLE療癒相談室帶著你，\n透過「西方占星術」與「東方紫微斗數」...,1980.0,NaN,3.0,2021/12/29,472.0,09,https://www.pressplay.cc/project/B470FFB794729...,https://static.pressplay.cc/static/uploads/pro...,NaN
3,pressplay_4,跟世界甜點冠軍學烘焙 - 在家做法式丹麥麵包！,NaN,跟著世界甜點冠軍Alexis老師做出招牌丹麥麵包！\n零烘焙基礎也可以做出好吃的法式丹麥麵包...,3600.0,NaN,4.0,2022/01/07,251.0,09,https://www.pressplay.cc/project/2EB38014AB607...,https://static.pressplay.cc/static/uploads/pro...,NaN
4,pressplay_5,主廚的 27 道私藏甜點秘笈 | 經典 x 台味一次上手,NaN,線上甜點教學課程，10 位甜點主廚，教導 27 道私藏甜點 + 3 種以上水果果醬，以台灣食...,6150.0,NaN,5.0,8/27-9/06 陸續抽出各獎項。抽獎後，將於此頁面中公布名單，並寄信給該中獎,4715.0,09,https://www.pressplay.cc/project/4F0C5A0FBABB4...,https://static.pressplay.cc/static/uploads/pro...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,pressplay_720,如何挖掘董事長概念股,NaN,一間公司的營運績效往往都是公司高層最清楚明瞭\n從產業趨勢、公司技術核心、財務狀況及未來的營...,13999.0,NaN,720.0,NaN,17.0,07,https://www.pressplay.cc/project/F387AC3D2166A...,https://static.pressplay.cc/static/uploads/pro...,NaN
720,pressplay_721,【脫單超簡單】認識到交往全方位教學,NaN,(一)\t為什麼會聊天這麼重要？\n(二)\t開始聊天前該知道的事\n(三)\t如何建立個人...,0.0,NaN,721.0,NaN,1.0,09,https://www.pressplay.cc/project/2A132C8C5B33C...,https://static.pressplay.cc/static/uploads/pro...,NaN
721,pressplay_722,記憶標本-昆蟲羊毛氈,NaN,羊毛氈怎麼做才不會失敗呢？四種羊毛氈教學，讓你快速掌握羊毛氈技巧。療癒的羊毛氈手工藝，在家也...,1800.0,NaN,722.0,NaN,141.0,09,https://www.pressplay.cc/project/35B81274FA6B4...,https://static.pressplay.cc/static/uploads/pro...,NaN
722,pressplay_723,技術分析補習班,NaN,技術分析大同小異，沒有時時刻刻複習觀察，如何找出差異性。在這裡直接給你差異性分析比較，讓你省...,1000.0,NaN,723.0,NaN,8.0,07,https://www.pressplay.cc/project/36E5E7753CB40...,https://static.pressplay.cc/static/uploads/pro...,NaN


In [46]:
df_teachers

,t_id,t_name,t_introduction
1,2.0,Aaron 張錫源,NaN
2,3.0,ELLE TAIWAN,NaN
3,4.0,Alexis Bouillet,NaN
4,5.0,河床法式甜點 黃偈,NaN
5,6.0,浦惠投顧,NaN
...,...,...,...
719,720.0,Allan冷哥,NaN
720,721.0,Dr.Love,NaN
721,722.0,Tigermilk,NaN
722,723.0,十年磨一劍,NaN


In [47]:
df_fundings

,f_id,f_due_date,f_process,c_id
1,NaN,倒數 2 天 18 小時 12 分 52 秒,17.76,pressplay_2
2,NaN,倒數 13 天 18 小時 13 分 46 秒,9.44,pressplay_3
3,NaN,倒數 2 天 18 小時 23 分 37 秒,8.36,pressplay_4
6,NaN,倒數 2 天 18 小時 12 分 19 秒,11.00,pressplay_7
8,NaN,倒數 2 天 18 小時 23 分 7 秒,5.56,pressplay_9
17,NaN,倒數 28 天 18 小時 12 分 5 秒,1.22,pressplay_18
50,NaN,倒數 22 天 18 小時 7 分 23 秒,1.22,pressplay_51
174,NaN,倒數 28 天 17 小時 53 分 0 秒,1.22,pressplay_175
195,NaN,倒數 298 天 16 小時 56 分 40 秒,1.22,pressplay_196
396,NaN,倒數 28 天 17 小時 29 分 11 秒,1.22,pressplay_397
